<a href="https://colab.research.google.com/github/pawel0508/MachineLearningBootcamp_I/blob/master/Przygotowanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn
import numpy as np
import pandas as pd
sklearn.__version__

'0.22.2.post1'

In [11]:
data = {
    'size': ['XL', 'L', 'M', np.nan, 'L', 'M'],
    'color': ['red', 'green', 'blue', 'green', 'red', 'green'],
    'gender': ['female', 'male', np.nan, 'male', 'female', 'female'],
    'price': [199.0, 89.0, np.nan, 99.0, 129.0, 79.0],
    'weight': [500, 450, 300, np.nan, 380, np.nan],
    'bought': ['yes', 'no', 'yes', 'no', 'yes', 'no']
}

df_raw = pd.DataFrame(data = data)
df_raw

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,NaN,NaN,300.0,yes
3,NaN,green,male,99.0,NaN,no
4,L,red,female,129.0,380.0,yes
5,M,green,female,79.0,NaN,no


Tworzenie kopii danych

In [12]:
df = df_raw.copy()


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   6 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  4 non-null      float64
 5   bought  6 non-null      object 
dtypes: float64(2), object(4)
memory usage: 416.0+ bytes


In [14]:
print(df.isnull().sum())
print(df.isnull().sum().sum())
df.isnull().sum()/len(df)



size      1
color     0
gender    1
price     1
weight    2
bought    0
dtype: int64
5


size      0.166667
color     0.000000
gender    0.166667
price     0.166667
weight    0.333333
bought    0.000000
dtype: float64

In [ ]:
for col in df.select_dtypes(include=['object']):
  df[col] = df[col].astype('category')

df['weight'] = df['weight'].astype('float')

In [ ]:
print(df.describe().T)
print(df.describe(include = ['category']).T)

        count   mean        std    min    25%    50%    75%    max
price     5.0  119.0  48.476799   79.0   89.0   99.0  129.0  199.0
weight    5.0  408.0  75.299402  300.0  380.0  410.0  450.0  500.0
       count unique     top freq
size       5      3       M    2
color      5      3     red    2
gender     5      2  female    3
bought     5      2     yes    3


Checmy etykiety przemapować na 0-1

In [ ]:
from sklearn.preprocessing import LabelEncoder
# LabelEncoder przeznaczony jest do przekształcania zmiennych docelowych (targetowych)
# do przekształacania zmiennych objaśniających jest inne narzędzie

le = LabelEncoder() # tworzymy instancję klasy przypisując ją do zmiennej
le.fit(df['bought']) # fit dopasowuje nasz encoder do zmiennej
le.transform(df['bought']) # transform dokonuje transformacji zmiennej na 0-1

array([1, 0, 1, 0, 1])

In [ ]:
# fit oraz transform mogą być stosowane w jednym kroku
le.fit_transform(df['bought'])

array([1, 0, 1, 0, 1])

In [ ]:

# czasami mamy więcej etykiet więc dobrze jest sobie sprawdzić jak mapowanie wygląda
le.classes_ # 'no' jest na miejscu 0, 'yes' jest na miejscu 1

array(['no', 'yes'], dtype=object)

In [ ]:
df['bought'] = le.fit_transform(df['bought'])

In [ ]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,green,male,89.0,450.0,0
2,M,blue,male,99.0,300.0,1
3,L,green,female,129.0,380.0,0
4,M,red,female,79.0,410.0,1


In [ ]:
# metoda inverse_transform pozwala wrócić do oryginalnej postaci
le.inverse_transform(df['bought'])

array(['yes', 'no', 'yes', 'no', 'yes'], dtype=object)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohn = OneHotEncoder(sparse = False)
ohn.fit(df[['size']]) # sparse = False, zwróci nam macierz, 
# oszczęność pamięci bo pamieta tylko gdzie jest jedynka
ohn.fit(df[['size']])
ohn.transform(df[['size']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [ ]:
ohn.categories_

[array(['L', 'M', 'XL'], dtype=object)]

In [ ]:
ohn = OneHotEncoder(sparse = False, drop = 'first') # usuwamy jedną kategorie (referencyjna)
# ponieważ zostawienie jej powodowałoby złe uczenie się modelu (współliniowość czy coś takiego)
ohn.fit(df[['size']])
ohn.transform(df[['size']])

array([[0., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.]])

Lepszym rozwiązaniem może być funkcja get_dummies()

In [ ]:
pd.get_dummies(df)

,price,weight,bought,size_L,size_M,size_XL,color_blue,color_green,color_red,gender_female,gender_male
0,199.0,500.0,1,0,0,1,0,0,1,1,0
1,89.0,450.0,0,1,0,0,0,1,0,0,1
2,99.0,300.0,1,0,1,0,1,0,0,0,1
3,129.0,380.0,0,1,0,0,0,1,0,1,0
4,79.0,410.0,1,0,1,0,0,0,1,1,0


In [ ]:
pd.get_dummies(df, drop_first=True) # o tym drop first pamiętać 

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,199.0,500.0,1,0,1,0,1,0
1,89.0,450.0,0,0,0,1,0,1
2,99.0,300.0,1,1,0,0,0,1
3,129.0,380.0,0,0,0,1,0,0
4,79.0,410.0,1,1,0,0,1,0


In [ ]:
pd.get_dummies(df, drop_first=True, prefix = 'new') # prefix oznacza od czego bedzie zaczynać się nasz kolumna

,price,weight,bought,new_M,new_XL,new_green,new_red,new_male
0,199.0,500.0,1,0,1,0,1,0
1,89.0,450.0,0,0,0,1,0,1
2,99.0,300.0,1,1,0,0,0,1
3,129.0,380.0,0,0,0,1,0,0
4,79.0,410.0,1,1,0,0,1,0


In [ ]:
pd.get_dummies(df, drop_first=True, prefix_sep = '-')

,price,weight,bought,size-M,size-XL,color-green,color-red,gender-male
0,199.0,500.0,1,0,1,0,1,0
1,89.0,450.0,0,0,0,1,0,1
2,99.0,300.0,1,1,0,0,0,1
3,129.0,380.0,0,0,0,1,0,0
4,79.0,410.0,1,1,0,0,1,0


In [ ]:
pd.get_dummies(df, drop_first=True, columns = ['size']) # kodujemy tylko jedną kolumnę 

,color,gender,price,weight,bought,size_M,size_XL
0,red,female,199.0,500.0,1,0,1
1,green,male,89.0,450.0,0,0,0
2,blue,male,99.0,300.0,1,1,0
3,green,female,129.0,380.0,0,0,0
4,red,female,79.0,410.0,1,1,0


Standaryzacja zmiennych liczbowych.
std() - pandas nieobciążony
std() - numpy obciążony

In [ ]:
print(f'Zmienna price:\n {df["price"]}')
print(f'Średnia price:\n {df["price"].mean():.2f}')
print(f'Odchylenie standardowe price:\n {df["price"].std():.2f}')
print(f'Standaryzacja price:\n {(df["price"]-df["price"].mean())/df["price"].std()}')

Zmienna price:
 0    199.0
1     89.0
2     99.0
3    129.0
4     79.0
Name: price, dtype: float64
Średnia price:
 119.00
Odchylenie standardowe price:
 48.48
Standaryzacja price:
 0    1.650274
1   -0.618853
2   -0.412568
3    0.206284
4   -0.825137
Name: price, dtype: float64


In [ ]:
def standardize(x):
  return (x - x.mean())/ x.std()
standardize(df['price'])

0    1.650274
1   -0.618853
2   -0.412568
3    0.206284
4   -0.825137
Name: price, dtype: float64

Gotowa funkcja służąca do standaryzacji jest zaimplementowana w sklearn.preprocessing i nazywa się scale()

In [ ]:
from sklearn.preprocessing import scale


In [ ]:
scale(df['price'])

array([ 1.84506242, -0.69189841, -0.4612656 ,  0.2306328 , -0.92253121])

Klasa StandardScaler pozwala zachować na przyszłość wartości statystyk, które pozwalają nam przetwarzać dane 

In [ ]:
from sklearn.preprocessing import StandardScaler 


In [ ]:
scaler = StandardScaler()
scaler.fit(df[['price']])
scaler.transform(df[['price']])

array([[ 1.84506242],
       [-0.69189841],
       [-0.4612656 ],
       [ 0.2306328 ],
       [-0.92253121]])

In [ ]:
scaler = StandardScaler()
df[['price', 'weight']] = scaler.fit_transform(df[['price', 'weight']])
df

,size,color,gender,price,weight,bought
0,XL,red,female,1.845062,1.366002,1
1,L,green,male,-0.691898,0.623610,0
2,M,blue,male,-0.461266,-1.603567,1
3,L,green,female,0.230633,-0.415740,0
4,M,red,female,-0.922531,0.029696,1


In [ ]:
df[['price']]

,price
0,1.845062
1,-0.691898
2,-0.461266
3,0.230633
4,-0.922531


In [ ]:
df = df_raw.copy()
le = LabelEncoder()
df['bought'] = le.fit_transform(df['bought'])

df[['price', 'weight']] = scaler.fit_transform(df[['price', 'weight']])
df = pd.get_dummies(df, drop_first = True)
df

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,1.845062,1.366002,1,0,1,0,1,0
1,-0.691898,0.623610,0,0,0,1,0,1
2,-0.461266,-1.603567,1,1,0,0,0,1
3,0.230633,-0.415740,0,0,0,1,0,0
4,-0.922531,0.029696,1,1,0,0,1,0


In [ ]:
target = pd.DataFrame(data = df.pop('bought'))

In [ ]:
target

,bought
0,1
1,0
2,1
3,0
4,1


,price,weight,size_M,size_XL,color_green,color_red,gender_male
0,1.845062,1.366002,0,1,0,1,0
1,-0.691898,0.623610,0,0,1,0,1
2,-0.461266,-1.603567,1,0,0,0,1
3,0.230633,-0.415740,0,0,1,0,0
4,-0.922531,0.029696,1,0,0,1,0
